<a href="https://colab.research.google.com/github/dalozgar/AO/blob/main/DAVID_ANGEL_LOZANO_GARCIA_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: DAVID ÁNGEL LOZANO GARCÍA <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. 

Los datos son:
Número de actores: 10
Número de tomas : 30
Actores/Tomas: https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario

                                        

# Adquisición y limpieza de los datos de entrada

 Antes que nada, procederemos a adquirir de la url facilitada el conjunto de datos de entrada, que habrá que limpiar y formatear adecuadamente para su posterior procesamiento.

In [46]:

import os
import pandas as pd


#public URL
#https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit#gid=0

#get spreadsheets key from url
gsheetkey = "1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0"

#sheet name
sheet_name = 'Hoja 1'

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
df = pd.read_excel(url,sheet_name=sheet_name, index_col=0)
display(df.head(5))

,Actor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
Toma,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,NaN,Total
1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0
2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0
3.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,3.0
4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,4.0


In [47]:
# Eliminamos filas superfluas
df.drop(['Toma','TOTAL'], inplace = True)

In [48]:
# Eliminamos columnas superfluas
# 'Unnamed: 11' : NaN

df.drop(['Unnamed: 11'], axis = 1, inplace = True)
df.drop(['Unnamed: 12'], axis = 1, inplace = True) # 'Unnamed: 12' : coste de la toma individualmente

In [49]:
# Eliminamos la última fila, que tiene índice y valores nulos
df = df[df.index.notnull()]
# display (df)

In [50]:
# Renombramos las columnas adecuadamente
df.columns = ['Actor_'+str(i) for i in range (1,len(df.columns)+1)]
# display (df)

In [51]:
import numpy as np
# Resetear el índice para que empiece en 1
df.index = np.arange(1, len(df) + 1)

In [52]:
# Comprobar el funcionamiento del slice a través de una máscara de índices
mask = [1,12,5,3]
df.loc[mask]

,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,Actor_10
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
12,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

  La única restricción del problema es que solamente puede haber 6 tomas por día. Eliminar esta restricción equivale a decir que pueden grabarse todas las tomas que se quiera por día, o sea que podrían grabarse todas en un día. Es decir, el problema sin restricciones admite una única solución, cuyo coste sería 10 ya que hay 10 actores y todos tendrían que venir ese día.

Teniendo en cuenta todas las restricciones, la cosa se complica. La restricción de rodar un máximo de seis tomas al día equivale a que habrá un mínimo de 5 días para grabar las 30 tomas. 
En tal caso, para recorrer todo el espacio de soluciones por un método de fuerza bruta, deberíamos considerar lo siguiente: el primer día podremos escoger 6 tomas de un total de 30 (equivale a combinaciones de 30 elementos tomados de 6 en 6, el segundo día 6 tomas de un total de 24, y así sucesivamente. Este planteamiento se resume en la siguiente tabla.

| DIA | TOMAS DISPONIBLES | POSIBILIDADES |
|-----|-------------------|---------------|
|  1  |       30          | $$ {30 \choose 6} $$  |
|  2  |       24          | $$ {24 \choose 6} $$  |
|  3  |       18          | $$ {18 \choose 6} $$  |
|  4  |       12          | $$ {12 \choose 6} $$  |
|  5  |       6          | $$ {6 \choose 6} =1 $$  |

Lo cual nos da un total de:
$$ n = {30 \choose 6} {24 \choose 6}{18 \choose 6}{12 \choose 6} = \frac{30!}{6! 24!} \frac{24!}{6! 18!} \frac{18!}{6! 12!} \frac{12!}{6! 6!} = \frac{30!}{(6!) ^5} = 1.3708741675893e18 $$

Es decir una cantidad inmanejable de posibilidades. 

Y sólo sería una cota inferior...

En efecto, cabe la posibilidad de que, dependiendo de los datos de la tabla de entreda, el número de posibles soluciones a considerar fuera aún mayor. Supongamos una situación como la siguiente (voy a considerar para este ejemplo una versión simplificada de nuestro problema con un total de 5 actores, 4 tomas por día y 8 tomas en total:

* SOLUCIÓN A: COSTE 7 (3+4) EN DOS SESIONES

| DIA | TOMAS | COSTE |
|-----|-------------------|---------------|
|  1  |       01010          |   |
|  1  |       01010          |   |
|  1  |       01010          |   |
|  1  |       00001          | 3  |
|  2  |       10101          |   |
|  2  |       10101          |   |
|  2  |       10101          |   |
|  2  |       00001          | 4  |

SOLUCIÓN B: COSTE 6 (2+3+1) EN TRES SESIONES

| DIA | TOMAS | COSTE |
|-----|-------------------|---------------|
|  1  |       01010          |   |
|  1  |       01010          |   |
|  1  |       01010          | 2 |
|  2  |       10101          |   |
|  2  |       10101          |   |
|  2  |       10101          |  3 |
|  3  |       00001          |   |
|  3  |       00001          |  1 |

Como vemos, la segunda distribución de tomas por días es de menor coste, ilustrando esta idea de que la solución óptima no tiene por qué corresponderse con una distribución de un máximo número de tomas en un mínimo número de días.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo).


Respuesta

En este problema, resultará útil identificar a las tomas por su índice en la tabla. De este modo, una sesión podrá describirse por un conjunto de números correspondientes a los índices de las tomas que forman parte de la sesión. Por ejemplo, si lo codificamos como una lista:

In [53]:
sesion_ejemplo = [4, 8, 15, 16, 23, 24]

# Podemos usar esa lista a modo de máscara y obtener un slice del dataframe
df.loc[sesion_ejemplo]

,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,Actor_10
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
23,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Una solución se modelaría así como un conjunto de sesiones. Por ejemplo como una lista de listas.

In [54]:
solucion_ejemplo_1 = [[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18],[19,20,21,22,23,24],[25,26,27,28,29,30]]
solucion_ejemplo_2 = [[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18],[19,20,21,22,23],[24],[25,26,27,28,29,30]]

El modelado mediante listas me ha dado buen resultado, pero tras revisar mi código y considerar posibles mejoras, pienso que es posible encontrar una forma mejor de codificar las soluciones mediante sets. Los sets tienen unas propiedades muy interesantes: no admiten valores repetidos (lo cual se adecúa a nuestro problema), y no son conjuntos ordenados. Esto podría aprovecharse para distinguir soluciones que en realidad son iguales pero que con el modelado mediante listas son procesadas como soluciones distintas, y con ello podría mejorarse el funcionamiento de mecanismos como la búsqueda tabú, por ejemplo.

Existe la dificultad de que Python no admite la construcción de sets de sets, pero esto podría solventarse mediante un set de frozensets. En tal caso, el tratamiento debería ser muy cuidadoso, ya que los frozensets son inmutables y el manejo se complica.

En definitiva, pienso que hay margen para la mejora.



Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La **función objetivo será el coste de la solución**, siendo la solución una asignación de las tomas a un conjunto de días. Se calculará, de acuerdo con las especificaciones del enunciado, de la siguiente forma: el coste de la solución será la suma de los costes de las sesiones que la conforman, y a su vez el coste de una sesión será la suma de los salarios de los actores que deben acudir ese día, independientemente del número de tomas en los que participen en esa sesión concreta.
El problema será, pues, de **minimización** de esa función objetivo.

Usaremos la siguiente función calcula_coste(solucion, df), que proporciona el coste conforme a lo anteriormente especificado. Para ilustrar fácilmente cómo funciona, se empleará una tercera solución de ejemplo (incompleta en este caso).



In [56]:
def calcula_coste(solucion, df):
        coste = 0
        for toma in solucion:
            df_i = df.loc[toma]        
            coste += df_i.any().sum()
        return(coste)


solucion_ejemplo_3 =[[1,2,3]]
ver_solucion (solucion_ejemplo_3, df)

print ("Para esta solución se ha obtenido un coste de:", 
       str(calcula_coste(solucion_ejemplo_3,df)))




* SESION 1:


,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,Actor_10
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


Para esta solución se ha obtenido un coste de: 6


Diseña un algoritmo para resolver el problema por fuerza bruta

# SOLUCIÓN POR FUERZA BRUTA
Como hemos avanzado se trata de un planteamiento inviable dada la ingente cantidad de posibilidades a considerar. No obstante, a efectos de ofrecer el algoritmo solicitado, se planteará dicha solución pero incorporando una limitación en el número de iteraciones para evitar tener que esperar un tiempo excesivo.

Por ponernos en situación: mi modesto ASUS F555L tiene un procesador de 2'11GHz. Supongamos que consiguiéramos un código tan sumamente optimizado que en un único ciclo del procesador procesara una solución. De este modo, el tiempo de ejecución sería:
$$ t_{exe} = \frac {1.37 \times 10^{18}}{2.11 \times 10^9} s \approx 20.5 años $$



La obtención sistemática y ordenada de las soluciones se llevará a cabo por medio del generador de soluciones que expongo a continuación.

In [57]:
import itertools 


def genera_solucion_fb (tomas, n):
    
    
    tomas_no_seleccionadas = tomas.copy()
    
    sesiones_1 = list(itertools.combinations(tomas_no_seleccionadas, n))
    
    for sesion_1 in sesiones_1:
        
        tomas_no_seleccionadas = [i for i in tomas if i not in sesion_1]
        sesiones_2 = list(itertools.combinations(tomas_no_seleccionadas, n))
        
        for sesion_2 in sesiones_2:
            tomas_no_seleccionadas = [i for i in tomas if i not in (sesion_1+sesion_2)]
            sesiones_3 = list(itertools.combinations(tomas_no_seleccionadas, n))
            
            for sesion_3 in sesiones_3:
                tomas_no_seleccionadas = [i for i in tomas if i not in (sesion_1+sesion_2+sesion_3)]
                sesiones_4 = list(itertools.combinations(tomas_no_seleccionadas, n))
                
                for sesion_4 in sesiones_4:
                    tomas_no_seleccionadas = [i for i in tomas if i not in (sesion_1+sesion_2+sesion_3+sesion_4)]
                    sesion_5 = tomas_no_seleccionadas
                    solucion = []
                    solucion.append(list(sesion_1))
                    solucion.append(list(sesion_2))
                    solucion.append(list(sesion_3))
                    solucion.append(list(sesion_4))
                    solucion.append(sesion_5)
                    yield (solucion) 
        
        

A continuación probamos nuestro iterador.

In [58]:
tomas = [i for i in range (1,len(df)+1)]
print (f"tomas ={tomas}")
i=0
for solucion in genera_solucion_fb(tomas, 6):
  
  print (f"{i} _: {solucion}")
  if (i>10):
    break
  i +=1

tomas =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
0 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]]
1 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 25], [24, 26, 27, 28, 29, 30]]
2 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 26], [24, 25, 27, 28, 29, 30]]
3 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 27], [24, 25, 26, 28, 29, 30]]
4 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 28], [24, 25, 26, 27, 29, 30]]
5 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 29], [24, 25, 26, 27, 28, 30]]
6 _: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 30], [24, 25, 26, 27, 2

El algoritmo por fuerza bruta se implementa en la siguiente función:

In [59]:
# Función que implementa el algoritmo por fuerza bruta
        
def fuerza_bruta_(df, max_tomas_dia, max_n = np.inf):
    tomas = [i for i in df.index]
    
    k=0
    mejor_coste = np.inf 
    mejor_solucion = None
    
    for solucion in genera_solucion_fb (tomas, max_tomas_dia):
        k+=1
        if (k==max_n): # Esto evita que se analicen todos los casos.
            break
        
        
        coste = calcula_coste (solucion,df)
        
        if (coste < mejor_coste):
            
            mejor_coste = coste
            mejor_solucion = solucion
            print (f"iter = {k} coste = {mejor_coste}")
            
    
    print (f"mejor_coste = {mejor_coste}\nmejor_solucion = {mejor_solucion}")
          
fuerza_bruta_(df,6,2000)

iter = 1 coste = 38
iter = 2 coste = 37
iter = 33 coste = 36
iter = 274 coste = 35
mejor_coste = 35
mejor_solucion = [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 21, 23, 24, 26, 29], [20, 22, 25, 27, 28, 30]]


Calcula la complejidad del algoritmo por fuerza bruta

**Respuesta**

 El número de operaciones para recorrer todo el espacio de soluciones mediante fuerza bruta es:

$$ T(n) = 3+ 16 \times \frac{n_{tomas}!}{(maxTomasPorDia!)^\frac{n_{tomas}}{maxTomasPorDia}}  $$

Es decir el problema es de complejidad $ O(n!) $

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

**Respuesta**

He implementado un algoritmo por recocido simulado. 
Adicionalmente he incorporado dentro de su bucle principal una búsqueda tabú que puede activarse por parametrización, aunque a decir verdad por las pruebas realizadas y el modelado y tipo de problema, no ha mejorado sustancialmente la calidad de las soluciones para el conjunto de datos del enunciado.

He recurrido a la OOP para construir la solución. Haré uso del siguiente objeto *organizador* que contiene las variables más importantes del problema y algunos métodos adicionales.

In [69]:

import random
import math  
# Objeto organizador que almacena los datos del problema

class organizador():
    
    def __init__(self, df):
        self.df = df
        self.temperatura = 0
        self.temp_0 = 0
        self.umbral_min_t = 0
        
        self.cooling_schedule = 0
        self.k = 0
        self.alpha = 2
        
        self.cooling_rate = 0.99
        
        self.mejor_solucion = []
        self.mejor_coste = np.inf
        
        self.inc_t = 0
        self.max_tomas_dia = np.inf

        self.tabu_search = False
        self.tabu_list = []
        self.tabu_size = np.inf
    
    def set_tabu_size (self,n):
        self.tabu_size = n
    
    def set_tabu_search (self,mode):
        self.tabu_search = mode
        
    def es_tabu (self,sol):
        repetido = False
        # print (f"___LISTA TABU___\n:{self.tabu_list}")
        for sol_tabu in self.tabu_list:
            if (np.array_equal(sol,sol_tabu)):
                
                print (f"-- INFO -- Iteración {self.k} \ttemperatura = {self.temperatura} \tDetectado por búsqueda tabú : SOLUCION REPETIDA")
                repetido = True
                
        return repetido

    def guarda_solucion (self,sol):
        
        self.tabu_list.append(sol)
        
        if (len(self.tabu_list) > self.tabu_size):
            self.tabu_list.pop(0) #elimina primer elemento (más antiguo)
    
    def set_temperatura (self, t):
        self.temperatura = t   
    
    def set_temperatura_0 (self, t):
        self.temp_0 = t
    
    def set_umbral_min_t (self, t):
        self.umbral_min_t = t
    
    def set_cooling_schedule (self, n):
        self.cooling_schedule = n
        
    def set_alpha (self, a):
        self.alpha = a
    
    def set_max_tomas_dia (self,n):
        self.max_tomas_dia = n
        
    def set_cooling_rate (self, rate):
        self.cooling_rate = rate
    

    #Funcion de probabilidad para aceptar peores soluciones
    def probabilidad(self, d):
        if random.random() <  math.exp( -1*d / self.temperatura)  :
            return True
        else:
            return False

    # Método para implementar el descenso de temperatura según el esquema de enfriamiento seleccionado
    def bajar_temperatura(self):
        
        modo = self.cooling_schedule
        
        if (modo ==0):
            # Criterio exponencial
            self.temperatura = self.cooling_rate * self.temperatura
        elif (modo ==1):
            # Criterio logaritmico multiplicativo
            self.temperatura = self.temp_0 / (1 + (self.alpha * math.log (1 + self.k)) )
           
        return self.temperatura



También usaré esta función, necesaria para la obtención de grupos (sesiones) aleatorios y que se usa dentro de la función de generación de soluciones vecinas.

In [70]:
# Función auxiliar que devuelve aleatoriamente un grupo alternativo 
# cuando se le pasa una solución y un grupo dentro de esa solución
def get_alternative_group (group, solucion): 
    random_group = group

    while (random_group == group):
        random_group = random.choice(solucion)

    return (random_group)

A continuación se aporta la función *genera_vecina* que genera una solución vecina a partir de otra aportada. El funcionamiento básico es el siguiente: de la solución de entrada se elegirá un grupo aleatorio, llamado grupo origen, del que se extraerá aleatoriamente uno de sus elementos (una toma). Se tomará un segundo grupo aleatoriamente, llamado grupo destino, y se le añadirá la toma extraída anteriormente.

Peculiaridades:


1.   Si el grupo origen es de 0 elementos, se eliminará de la solución, pasando ésta a tener una sesión menos. Esto se hace así porque el recocido simulado partirá de una solución trivial de tantas sesiones como tomas, y evidentemente conforme se vaya avanzando en la búsqueda de mejores soluciones, el número de sesiones irá disminuyendo.

2.   Si el grupo destino, tras la inserción del nuevo elemento que trasegó de un grupo a otro, contiene más elementos de los permitidos, se repetirá la operación tantas veces como sea necesario hasta que se cumpla la condición de tamaño de las sesiones.





In [74]:

import copy
# Funcion de generar vecinas
def genera_vecina(solucion, org): 
    n_sesiones = len (solucion)
    
    vecina = copy.deepcopy(solucion)

    random_group_origen = get_alternative_group (None, vecina) 

    random_group_destino = get_alternative_group (random_group_origen, vecina)

    # Grupo desaparece al quedarse sin elementos
    if (len(random_group_origen) ==0):
        vecina.remove(random_group_origen)
    else:

        elem = random.choice(random_group_origen)
        random_group_origen.remove(elem)
        random_group_destino.append(elem)
        
        
    while (len (random_group_destino) > org.max_tomas_dia):
        
        elem = random.choice(random_group_destino)
        random_group_destino.remove(elem)
        random_group_destino = get_alternative_group (random_group_destino, vecina)
        random_group_destino.append(elem)

    return (vecina)


A continuación, la función principal del recocido simulado, de una forma muy similar a la estudiada en clase.

In [72]:

#Función principal de recocido
def recocido_simulado(org):
    
    org.set_temperatura (org.temp_0)

    df = org.df

    # Solución trivial inicial
    sol_0 = [[i] for i in range (1, len(df)+1)] #cada toma se graba un dia distinto (máximo coste)
    print (sol_0)
    
    solucion_referencia = sol_0
    coste_referencia = calcula_coste(solucion_referencia,df)
  
    mejor_solucion = sol_0
    mejor_coste = coste_referencia
    
    # BUCLE PRINCIPAL
    while org.temperatura > org.umbral_min_t:   
        
        org.k +=1
        
        vecina = genera_vecina (solucion_referencia, org)
        
        # La búsqueda tabú es configurable en la inicialización
        if (org.tabu_search):
          buscar_vecina = True
          while (buscar_vecina):
              vecina = genera_vecina (solucion_referencia,org)
              buscar_vecina = org.es_tabu (vecina)
          org.guarda_solucion(vecina)

        coste_vecina = calcula_coste(vecina,df)

        # Si es la mejor solución de todas, se guarda
        if coste_vecina < mejor_coste:
            mejor_solucion = vecina
            mejor_coste = coste_vecina
            print (f"-- INFO -- Iteración {org.k} \ttemperatura = {org.temperatura}\t COSTE = {mejor_coste}")

        # Aquí se añade cierta aleatoriedad al algoritmo
        if coste_vecina < coste_referencia or org.probabilidad(abs(coste_referencia - coste_vecina) ) :
            solucion_referencia = vecina
            coste_referencia = coste_vecina
            
        #Bajamos la temperatura
        org.bajar_temperatura()
        
        
    org.mejor_solucion = mejor_solucion
    org.mejor_coste = mejor_coste

    return (mejor_solucion, mejor_coste, org.k)    




Y finalmente ya tenemos todas las herramientas necesarias para probar el algoritmo. Dado que he dejado gran parte de las variables como parámetros configurables, hay que ser cuidadoso antes de lanzar la rutina principal y configurar adecuadamente el objeto organizador.

In [76]:
# Creación del objeto
org = organizador(df)

##### PRUEBA DEL ALGORITMO #########

# Configuración del objeto organizador
temp = 100000 # Temperatura inicial
org.set_max_tomas_dia(6)
org.k = 0 # k es un contador de iteraciones

org.set_temperatura_0 (temp) 

# Configuración del esquema de enfriamiento
org.set_cooling_schedule(0) #0: exponencial
org.set_umbral_min_t (0.01)
org.set_cooling_rate (0.99)

# Prueba de otro esquema de enfriamiento
# org.set_cooling_schedule(1) #1: logaritmico multiplicativo
# org.set_alpha (1.25) # alpha > 1
# org.set_umbral_min_t (1)


# Configuración de la búsqueda tabú
org.set_tabu_search(True)
org.set_tabu_size(20)

# Simulated Annealing
res = recocido_simulado (org)

print (f"** MEJOR COSTE = {res[1]}\nMEJOR SOLUCION = {res[0]} en {res[2]} iteraciones")

[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]]
-- INFO -- Iteración 1 	temperatura = 100000	 COSTE = 93
-- INFO -- Iteración 2 	temperatura = 99000.0	 COSTE = 91
-- INFO -- Iteración 4 	temperatura = 97029.9	 COSTE = 89


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


-- INFO -- Iteración 8 	temperatura = 93206.53479069899	 COSTE = 88
-- INFO -- Iteración 10 	temperatura = 91351.72474836408	 COSTE = 86
-- INFO -- Iteración 14 	temperatura = 87752.10229989678	 COSTE = 84
-- INFO -- Iteración 17 	temperatura = 85145.77710948754	 COSTE = 83
-- INFO -- Iteración 19 	temperatura = 83451.37614500873	 COSTE = 82
-- INFO -- Iteración 25 	temperatura = 78567.81408072186	 COSTE = 80
-- INFO -- Iteración 26 	temperatura = 77782.13593991465	 COSTE = 79
-- INFO -- Iteración 28 	temperatura = 76234.27143471033	 COSTE = 77
-- INFO -- Iteración 32 	temperatura = 73230.33696543971	 COSTE = 76
-- INFO -- Iteración 36 	temperatura = 70344.76949995689	 COSTE = 75
-- INFO -- Iteración 38 	temperatura = 68944.90858690774	 COSTE = 74
-- INFO -- Iteración 44 	temperatura = 64910.26283684018	 COSTE = 72
-- INFO -- Iteración 47 	temperatura = 62982.363120323185	 COSTE = 71
-- INFO -- Iteración 48 	temperatura = 62352.539489119954	 COSTE = 68
-- INFO -- Iteración 49 	temperat

(*)Calcula la complejidad del algoritmo 

Para calcular el número de iteraciones en el bucle principal del recocido simulado, y suponiendo un criterio de enfriamiento exponencial, hay que tener en cuenta las temperaturas final e inicial y la tasa de enfriamiento. De modo que:
REVISAR ESTO
$$ \alpha^n \times T_0 \lt T_f $$
$$ \alpha^n \lt \frac{T_f}{T_0} $$
$$ n \times \log \alpha \gt \log \frac{T_f}{T_0} $$
$$ n   \gt \frac{\log \frac{T_f}{T_0}}{\log \alpha} $$

Para los datos de nuestro ejemplo, 

$$ n   \gt \frac{\log \frac{0.01}{100000}}{\log 0.99} = 1603.73  $$

Sabiendo que hay $ \frac{\log \frac{T_f}{T_0}}{\log \alpha}  $ iteraciones, el número de operaciones del algoritmo será:

$$ T(n) = 10 +  \frac{\log \frac{T_f}{T_0}}{\log \alpha} \times (21 + \frac {n_{tomas}}{maxTomasPorDia}) $$


Es decir, el problema tiene una complejidad aproximadamente lineal $ O(n) $ con respecto al número de tomas, pero condicionada por los valores de $ T_0 $, $ T_f $ y $ \alpha $. En cierto modo podemos decir que la complejidad aumenta, lógicamente, cuanto más intensa sea la búsqueda.

A la vista de la complejidad del algoritmo, vemos que es mejor que el método de fuerza bruta, pues mediante una búsqueda inteligente podemos obtener unos resultados bastante buenos en cuestión de segundos, sin que merezca la pena analizar todas las posibles combinaciones, lo cual requiere un tiempo de ejecución muy elevado. Por contra, no tenemos garantía de que estemos alcanzando el óptimo global.


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Obtengo un juego de datos de entrada aleatorios a partir de las funciones random de numpy.

In [ ]:
n_tomas =100
n_actores =50
data = np.random.randint(2, size=n_tomas*n_actores).reshape(n_tomas,n_actores)
df_random = pd.DataFrame(data=data)
df_random.index = np.arange(1, len(df_random) + 1)
# display  (df)

Aplica el algoritmo al juego de datos generado

Respuesta

In [ ]:
org_2 = organizador(df_random)


##### PRUEBA DEL ALGORITMO #########
temp = 1000000
org_2.k = 0

org_2.set_temperatura_0 (temp)

org_2.set_cooling_schedule(0) #0: exponencial
org_2.set_umbral_min_t (0.01)
org_2.set_cooling_rate (0.99)


# org.set_cooling_schedule(1) #1: logaritmico multiplicativo
# org.set_alpha (1.25) # alpha > 1
# org.set_umbral_min_t (1)


org_2.set_max_tomas_dia(5)

org_2.set_tabu_search(False)
org_2.set_tabu_size(20)

res = recocido_simulado (org_2)

print (f"** MEJOR COSTE = {res[1]}\nMEJOR SOLUCION = {res[0]} en {res[2]} iteraciones")

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

# REFERENCIAS
1. [Artículo: "A Comparison of Cooling Schedules for Simulated Annealing (Artificial Intelligence)" ](http://what-when-how.com/artificial-intelligence/a-comparison-of-cooling-schedules-for-simulated-annealing-artificial-intelligence/))

        
2. Proyecto Fin de Máster: [Aplicación de algoritmos heurísticos para optimizar el coste de doblaje de películas](http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_759.pdf)



Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño


Como comenté antes, el modelo de datos podría depurarse haciendo uso de sets de Python en lugar de listas, permitiendo así una verificación de soluciones duplicadas más exhaustiva con la búsqueda tabú.

Por las pruebas realizadas, se deduce que el esquema de enfriamiento utilizado inflye en la calidad de las soluciones encontradas. En mi código, sólo he implementado los criterios exponencial y logarítmico multiplicativo, pero el método bajar_temperatura puede adaptarse para otros esquema (ver apartado de referencias). La forma más o menos pronunciada de la curva, especialmente durante los primeros momentos del enfriamiento, determina que se diversifique más o menos la búsqueda por rutas de búsqueda subóptimas al aceptar peores soluciones de referencia con mayor probabilidad. Para los dos casos implementados, se observa que el criterio logarítmico-multiplicativo desciende muy rápidamente al principio, aunque luego tarda mucho más que la exponencial en converger asintóticamente al valor mínimo de temperatura establecido como condición de parada. Intuyo que probablemente algunos esquemas serán mejores o peores que otros en función del problema particular a que nos enfrentemos. En el caso que nos ocupa, el criterio exponencial resulta más sencillo, de convergencia más rápida y con mejores resultados, pero sería interesante experimentar con otros esquemas.

En cuanto a modificaciones del problema, resultaría interesante añadir el coste por sesiones, es decir, el coste de tener ocupado un día el estudio de grabación. También podría considerarse, en un caso más realista, que los dobladores no cobran todos igual, por ejemplo. Esto obligaría a realizar modificaciones en el modelado de los datos para incluir esta información, y en la función calcula_coste.